# Battle of the Neighborhoods
## Coursera Capstone Project

In [68]:
import pandas as pd
import numpy as np

## We make use of the data available on Wikipedia. 
Here we are using the link to the Wikipedia page hat has information about the various postal codes, Boroughs, and neighborhoods in Toronto. We will import this data into the Jupyter Notebook.

In [69]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

## We are trying to import the Table from the Wikipedia page using Beautiful Soup

In [70]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")


We know by inspecting the Wikipedia page that our desired table has <b>class = "wikitable sortable"</b><br>
We can use this information to select our required table and store it in right_table.

In [71]:
right_table=soup.find('table', class_='wikitable sortable')


## Converting into Pandas dataframe: 
Once we have our table stored in <b>right_table</b>, we can extract the columns and cell values from this table<br>
We know that each row begins with a <b>tr</b> tab and each cell value will begin with a <b>td</b> tab.<br>
using this information we can go through the table and extract the values one by one and store them into 3 columns which we can later rename when we add them into our pandas dataframe.

In [72]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

We now rename our columns and add them to our pandas data frame names <b>df</b><br>
The three columns are names <b>Postal Code</b>, <b>Borough</b> and <b>Neighborhood</b>

In [73]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#  Cleaning the data
We have to process the data so that we have the data  frame in the required  format

## We only process the cells that have an assigned borough. Ignore cells with a borough that is <b>Not assigned</b>.

In [74]:
df = df[df["Borough"]!='Not assigned\n']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [75]:
df_combined = df.groupby("Postal Code", sort=False).agg( ','.join)
df_combined.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York\n,Parkwoods\n
M4A,North York\n,Victoria Village\n
M5A,Downtown Toronto\n,"Regent Park, Harbourfront\n"
M6A,North York\n,"Lawrence Manor, Lawrence Heights\n"
M7A,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


We have to remove the '\n' character from the end of the Borough and Neighborhood column.<br>
We do this by splitting the data and reassigning the column values accordingly.

In [76]:
B = pd.DataFrame(df_combined.Borough.str.split('\n').str[0])
N = pd.DataFrame(df_combined.Neighborhood.str.split('\n').str[0])
df_combined['Borough'] = B["Borough"]
df_combined['Neighborhood'] = N["Neighborhood"]
df_combined.reset_index(inplace = True)
df_combined.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [77]:
df[df["Neighborhood"]=='Not assigned']

,Postal Code,Borough,Neighborhood


From the above result, it is clear that there is no such neighborhood that is not assigned, but has a Borough. 

## We check the shape of the dataframe

In [78]:
df_combined.shape

(103, 3)

<b>The cleaned Data Set has 103 rows and 3 columns. </b>

We Sort the data based on Postal Code so that we can combine this data with its Latitudes and Longitudes

In [79]:
df_combined.sort_values(by = ["Postal Code"],inplace = True)

In [80]:
df_combined.reset_index(inplace=True)
df_combined.head(5)

,index,Postal Code,Borough,Neighborhood
0,6,M1B,Scarborough,"Malvern, Rouge"
1,12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,22,M1G,Scarborough,Woburn
4,26,M1H,Scarborough,Cedarbrae


In [81]:
# The code was removed by Watson Studio for sharing.

## Adding the Latitudes and Longitudes
We load the data into Location data frame and then combine it with our data to complete our data frame

In [82]:
Location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [83]:
df_combined["Latitude"] = Location["Latitude"]
df_combined["Longitude"] = Location["Longitude"]
df_combined.head()

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,22,M1G,Scarborough,Woburn,43.770992,-79.216917
4,26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [84]:
df_combined.reset_index(inplace=True)

In [85]:
df_combined.head()

,level_0,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,22,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [86]:
data = df_combined[["Postal Code","Borough","Neighborhood","Latitude","Longitude"]]
data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


This is our cleaned data frame ready to use for analysis.